In [2]:
import os
import SimpleITK as sitk
import numpy as np
import mclahe as mc
from pathlib import Path
from skimage import exposure
import pandas as pd

In [4]:

path = '../ICM_LGE_clin_myo_mask_labeled_filtered_3y_clahe_0.1'

for data_set in os.listdir(path):
    for patient in os.listdir(os.path.join(path, data_set)):
        id = ('_').join(patient.split("_")[0:3])
        src_patient_folder = Path(os.path.join(path, data_set, patient))
        image_path = os.path.join(src_patient_folder, f"LGE_{id}.nii.gz")
        img = sitk.ReadImage(image_path)
        img_array = sitk.GetArrayFromImage(img)
        img_array = img_array.astype(np.float32)
        kernel_size = (img_array.shape[0], 8, 8)
        clip_limit=0.1
        clahe_image = mc.core.mclahe(img_array, n_bins=256, clip_limit=clip_limit, kernel_size=kernel_size)
        # normalized_array = exposure.equalize_adapthist(img_array, clip_limit=0.03)
        # normalized_sitk = sitk.GetImageFromArray(normalized_array)
        # normalized_sitk.CopyInformation(img)
        # output_path = os.path.join(src_patient_folder, f'ahe_LGE_{id}.nii.gz')
        # sitk.WriteImage(normalized_sitk, output_path)
        # print(clahe_img.shape)
        clahe_image = sitk.GetImageFromArray(clahe_image)
        clahe_image = sitk.Cast(clahe_image, sitk.sitkFloat32)
        clahe_image.CopyInformation(img)

        # Save preprocessed image as NIfTI
        output_path = os.path.join(src_patient_folder, f'LGE_{id}.nii.gz')
        sitk.WriteImage(clahe_image, output_path)

In [3]:


path = '../LGE_CINE_clin_myo_clahe_0.005_masked'
threshold_value = 0.05
kernel_size=(10, 25, 25)

for data_set in os.listdir(path):
    for patient in os.listdir(os.path.join(path, data_set)):
        id = patient.split("_")[0]
        src_patient_folder = Path(os.path.join(path, data_set, patient))
        image_path = os.path.join(src_patient_folder, f"LGE_{id}.nii.gz")
        myoseg_f = os.path.join(src_patient_folder, f"MYO_{id}.nrrd")
        img = sitk.ReadImage(image_path)
        myoseg = sitk.ReadImage(str(myoseg_f))

        binary_mask = sitk.BinaryThreshold(myoseg, lowerThreshold=threshold_value, upperThreshold=1.0, insideValue=1, outsideValue=0)

        # Create a SimpleITK structuring element
        structuring_element = sitk.sitkBox
        closing_filter = sitk.BinaryMorphologicalClosingImageFilter()
        closing_filter.SetKernelType(structuring_element)
        closing_filter.SetKernelRadius(kernel_size)
        
        # Perform 3D morphological closing on the integer image
        result_stack = closing_filter.Execute(binary_mask)

        # # Convert the result back to the original floating-point type if needed
        # result_stack.SetSpacing(img.GetSpacing())
        # result_stack.SetOrigin(img.GetOrigin())
        # result_stack.SetDirection(img.GetDirection())
        result_stack.CopyInformation(img)

        masked_image = sitk.Mask(img, result_stack)


        # Save preprocessed image as NIfTI
        # output_path_image = os.path.join(src_patient_folder, f'LGE_{id}.nii.gz')
        output_path_mask = os.path.join(src_patient_folder, f'MASK_{id}.nii.gz')
        # sitk.WriteImage(masked_image, output_path_image)
        sitk.WriteImage(result_stack, output_path_mask)

In [7]:
df = pd.read_csv('../beta_search.csv')

df.head(n=20)

,Unnamed: 0,KL_Loss,Reconstruction_Loss,training_loss
0,0.000001,"metatensor(7.5528e-07, device='cuda:0', grad_f...","metatensor(9.6543e+08, device='cuda:0', grad_f...","metatensor(9.6543e+08, device='cuda:0', grad_f..."
1,0.000010,"metatensor(1.0152e-05, device='cuda:0', grad_f...","metatensor(8.7718e+08, device='cuda:0', grad_f...","metatensor(8.7718e+08, device='cuda:0', grad_f..."
2,0.000100,"metatensor(0.0002, device='cuda:0', grad_fn=<A...","metatensor(8.6291e+08, device='cuda:0', grad_f...","metatensor(8.6291e+08, device='cuda:0', grad_f..."
3,0.001000,"metatensor(0.0030, device='cuda:0', grad_fn=<A...","metatensor(8.5586e+08, device='cuda:0', grad_f...","metatensor(8.5586e+08, device='cuda:0', grad_f..."
4,0.010000,"metatensor(0.0793, device='cuda:0', grad_fn=<A...","metatensor(8.5141e+08, device='cuda:0', grad_f...","metatensor(8.5141e+08, device='cuda:0', grad_f..."
5,0.100000,"metatensor(2.9178, device='cuda:0', grad_fn=<A...","metatensor(8.3734e+08, device='cuda:0', grad_f...","metatensor(8.3734e+08, device='cuda:0', grad_f..."
6,1.000000,"metatensor(69.4446, device='cuda:0', grad_fn=<...","metatensor(8.1060e+08, device='cuda:0', grad_f...","metatensor(8.1060e+08, device='cuda:0', grad_f..."
7,10.000000,"metatensor(1047.0236, device='cuda:0', grad_fn...","metatensor(7.8483e+08, device='cuda:0', grad_f...","metatensor(7.8483e+08, device='cuda:0', grad_f..."
8,100.000000,"metatensor(13417.8398, device='cuda:0', grad_f...","metatensor(7.7723e+08, device='cuda:0', grad_f...","metatensor(7.7724e+08, device='cuda:0', grad_f..."
9,1000.000000,"metatensor(121360.0859, device='cuda:0', grad_...","metatensor(7.5488e+08, device='cuda:0', grad_f...","metatensor(7.5500e+08, device='cuda:0', grad_f..."
